Importing all the necessary dependencies

In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

2022-06-25 22:45:10.731842: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


Importing the dataset

In [61]:
rawDataset = pd.read_csv('/home/rumbleftw/Documents/Codes/global-censorship/datasets/text/finalData.csv')
rawDataset = rawDataset[:10000]

In [62]:
rawDataset['text'] = rawDataset['text'].values.astype('str')

Tokenizing the word vocab

In [63]:
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.oov_token = '<shobdobhandarErBahire>'
tokenizer.fit_on_texts(rawDataset['text'])
vocab = tokenizer.word_index
vocabSize = len(vocab)
vocabSize, vocab

(34775,
 {'<shobdobhandarErBahire>': 1,
  'the': 2,
  'to': 3,
  'of': 4,
  'a': 5,
  'and': 6,
  'you': 7,
  'i': 8,
  'is': 9,
  'that': 10,
  'in': 11,
  'it': 12,
  'this': 13,
  'for': 14,
  'on': 15,
  'not': 16,
  'be': 17,
  'are': 18,
  'as': 19,
  'have': 20,
  'your': 21,
  'if': 22,
  'with': 23,
  'aicle': 24,
  'was': 25,
  'but': 26,
  'or': 27,
  'my': 28,
  'page': 29,
  'an': 30,
  'from': 31,
  'by': 32,
  'so': 33,
  'do': 34,
  'at': 35,
  'about': 36,
  'me': 37,
  'can': 38,
  'wikipedia': 39,
  'what': 40,
  'has': 41,
  'will': 42,
  'all': 43,
  'there': 44,
  'would': 45,
  'please': 46,
  'like': 47,
  'they': 48,
  'just': 49,
  'its': 50,
  'no': 51,
  'one': 52,
  'dont': 53,
  'talk': 54,
  'any': 55,
  'been': 56,
  'he': 57,
  'we': 58,
  'more': 59,
  'should': 60,
  'which': 61,
  'some': 62,
  'other': 63,
  'who': 64,
  'here': 65,
  'im': 66,
  'also': 67,
  'think': 68,
  'know': 69,
  'his': 70,
  'how': 71,
  'see': 72,
  'out': 73,
  'because'

Testing the tokenized words

Sequencing the whole dataset and padding to uniform shape

In [64]:
textData = tokenizer.texts_to_sequences(rawDataset['text'])
paddedTextData = tf.keras.preprocessing.sequence.pad_sequences(textData, padding='pre')
paddedTextData.shape, paddedTextData

((10000, 1000),
 array([[    0,     0,     0, ...,    17,   667,    78],
        [    0,     0,     0, ...,    21,  4399,   880],
        [    0,     0,     0, ...,    15, 10647,  4401],
        ...,
        [    0,     0,     0, ...,   564,  4290, 34772],
        [    0,     0,     0, ...,     3,   174,    13],
        [    0,     0,     0, ...,  1480,    85,   706]], dtype=int32))

Splitting into training and testing datasets

In [66]:
trainLength = 8000


xTrain = paddedTextData[:trainLength]
yTrain = rawDataset['is_offensive'].to_numpy()[:trainLength]

xTest = paddedTextData[trainLength:]
yTest = rawDataset['is_offensive'].to_numpy()[trainLength:]

In [67]:
yTest.shape, xTest.shape

((2000,), (2000, 1000))

Defining the model

In [71]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Embedding(vocabSize, 64, input_length=1000))
model.add(tf.keras.layers.GlobalAveragePooling1D())
model.add(tf.keras.layers.Dense(32, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

Training the model

In [72]:
model.fit(xTrain, yTrain, epochs=5)

Epoch 1/5
250/250 [==============================] - 19s 69ms/step - loss: 0.5596 - accuracy: 0.7987
Epoch 2/5
250/250 [==============================] - 17s 67ms/step - loss: 0.4981 - accuracy: 0.7920
Epoch 3/5
250/250 [==============================] - 18s 70ms/step - loss: 0.4466 - accuracy: 0.8085
Epoch 4/5
250/250 [==============================] - 17s 68ms/step - loss: 0.4184 - accuracy: 0.8070
Epoch 5/5
250/250 [==============================] - 18s 71ms/step - loss: 0.3879 - accuracy: 0.8113


Evaluating on testing dataset

In [81]:
model.evaluate(xTest[:100], yTest[:100])

4/4 [==============================] - 0s 3ms/step - loss: 0.3426 - accuracy: 0.9000


[0.34257325530052185, 0.8999999761581421]

Real life testing

In [82]:
texts = ["We are doing our project", "you are a piece of shit", "you bloody bitch", "good night brother"]

sequencedText = tokenizer.texts_to_sequences(texts)
print(sequencedText)
paddedText = tf.keras.preprocessing.sequence.pad_sequences(sequencedText, padding='pre')
print(paddedText)
model.predict(paddedText)

[[58, 18, 264, 170, 507], [7, 18, 5, 893, 4, 430], [7, 3286, 110], [99, 1038, 1673]]
[[   0   58   18  264  170  507]
 [   7   18    5  893    4  430]
 [   0    0    0    7 3286  110]
 [   0    0    0   99 1038 1673]]


array([[1.2183372e-11],
       [9.9992746e-01],
       [1.0000000e+00],
       [1.2749022e-01]], dtype=float32)